## Tokenized news를 TFIDF로 벡터화

In [1]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth
import json
import os
from collections import defaultdict
import matplotlib.pyplot as plt
from gensim.models.word2vec import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale
from sklearn.metrics import silhouette_samples, silhouette_score

C:\Users\dpqls\anaconda3\envs\npl\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\dpqls\anaconda3\envs\npl\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
tmp_df = pd.read_csv('../data/tokenized_news/tokenized_all')

In [3]:
tmp_df = tmp_df.drop(columns='Unnamed: 0')

In [4]:
tfidf_vect = TfidfVectorizer(max_df=0.90, min_df=0.40)

#opinion_text 컬럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(tmp_df['text_V'].apply(lambda x: np.str_(x)))

In [163]:
# from sklearn.cluster import KMeans

# # 5개 집합으로 군집화 수행. 
# # 예제를 위해 동일한 클러스터링 결과 도출용 random_state=0 
# km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
# km_cluster.fit(feature_vect)
# cluster_label = km_cluster.labels_
# cluster_centers = km_cluster.cluster_centers_

In [164]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화 (리뷰)
km_cluster = KMeans(n_clusters=3, init='k-means++', max_iter=10000)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_


# 소속 클러스터를 cluster_label 컬럼으로 할당하고 cluster_label 값으로 정렬
tmp_df['cluster_label'] = cluster_label
tmp_df.sort_values(by='cluster_label')

,title,text,url,media,date,title_V,text_V,cluster_label,silhouette_coeff
6587,열악한 환경에 잇단 사망 뒤에야…대남병원 격리환자 60명 옮긴다,보건당국이 경북 청도대남병원에 격리된 채 코로나19 치료를 받아온 5층 폐쇄(보호)...,https://news.naver.com/main/read.nhn?mode=LPOD...,한겨레,2020-02-28,열악한 환경에 잇단 사망 뒤에야…대남병원 격리환자 60명 옮긴다,"보건, 당국, 경북, 청도, 대남, 병원, 격리, 코로나, 치료, 층, 폐쇄, 보호...",0,0.447189
8227,<‘코로나19’ 초비상>유가 또 폭락… “50달러선 회복 내년에나 가능할 것”,美 ‘유럽발 입국금지’ 직격탄 \n\n브렌트유 하루새 2.57달러 ↓ \n\n세계경...,https://news.naver.com/main/read.nhn?mode=LPOD...,문화일보,2020-03-13,<‘코로나19’ 초비상>유가 또 폭락… “50달러선 회복 내년에나 가능할 것”,"유럽, 입국, 금지, 직격탄, 브렌, 트유, 하루, 달러, 세계, 경제, 위협, 뇌...",0,0.057389
8228,"<‘코로나19’ 초비상>EU “美, 일방적으로 입국금지” 비난… 트럼프 “유럽도 협...",대서양 동맹 다시 정면충돌\n\n도널드 트럼프 미국 대통령의 유럽 국가에 대한 30...,https://news.naver.com/main/read.nhn?mode=LPOD...,문화일보,2020-03-13,"<‘코로나19’ 초비상>EU “美, 일방적으로 입국금지” 비난… 트럼프 “유럽도 협...","대서양, 동맹, 다시, 정면, 충돌, 도널드, 트럼프, 미국, 대통령, 유럽, 국가...",0,0.245486
8229,<‘코로나19’ 초비상>실물경제 타격→증시 패닉…‘코로나發 금융위기’ 시작되나,- 코스피 장중 1700붕괴\n\n장중 외국인 5618억 등 투매\n\n장중 최저 ...,https://news.naver.com/main/read.nhn?mode=LPOD...,문화일보,2020-03-13,<‘코로나19’ 초비상>실물경제 타격→증시 패닉…‘코로나發 금융위기’ 시작되나,"코스피, 붕괴, 외국인, 투매, 최저, 하락, 사상, 최대, 낙폭, 공포, 지수, ...",0,0.514536
8230,<‘코로나19’ 초비상>금융위기 ‘선제대응’- 정부·韓銀 ‘정책공조’ 시급,"- 韓銀, 기준금리 인하 예고\n\n임시금통위 내주초 개최 유력\n\n부동산 상승 ...",https://news.naver.com/main/read.nhn?mode=LPOD...,문화일보,2020-03-13,<‘코로나19’ 초비상>금융위기 ‘선제대응’- 정부·韓銀 ‘정책공조’ 시급,"기준금리, 인하, 예고, 임시, 금통위, 주초, 개최, 유력, 부동산, 상승, 부작...",0,0.155037
...,...,...,...,...,...,...,...,...,...
19229,"쇠목줄이 풀렸을 때… 아홉살 소녀, 4층 난간 넘어 필사의 탈출",더위에도 온몸을 다 덮은 긴소매·긴바지. 손과 발엔 누군가가 지진 듯한 화상….\n...,https://news.naver.com/main/read.nhn?mode=LPOD...,국민일보,2020-06-12,"쇠목줄이 풀렸을 때… 아홉살 소녀, 4층 난간 넘어 필사의 탈출","더위, 온몸, 소매, 바지, 손, 누군가, 지진, 화상, 소녀, 높이, 층, 빌라,...",2,0.228149
6422,"21세기가 사랑하는 백석의 詩, 그 시작은 1930년 조선일보였다","인물과 사건으로 본 조선일보 100년\n\n\n\n\n""가난한 내가/ 아름다운 나타...",https://news.naver.com/main/read.nhn?mode=LPOD...,조선일보,2020-02-27,"21세기가 사랑하는 백석의 詩, 그 시작은 1930년 조선일보였다","인물, 사건, 나타, 샤를, 사랑, 오늘밤, 린다, 백석, 사진, 시인, 타샤, 당...",2,0.199569
19231,코로나에 경로당 폐쇄… 폭염 속 노인을 위한 ‘쉼터’는 없다,신종 코로나바이러스감염증(코로나19) 확산으로 경로당과 무더위 쉼터 운영이 중단되면...,https://news.naver.com/main/read.nhn?mode=LPOD...,국민일보,2020-06-12,코로나에 경로당 폐쇄… 폭염 속 노인을 위한 ‘쉼터’는 없다,"신종, 코로나바이러스, 감염증, 코로나, 확산, 경로당, 무더위, 운영, 중단, 노...",2,-0.017949
19216,"창녕 아동 학대, 달군 쇠젓가락까지…피해 아동 “학교엔 가고 싶다”",[경향신문] \n\n\n\n\n부모에게 모진 학대를 받은 경남 창녕의 초등 4학년생...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-06-12,"창녕 아동 학대, 달군 쇠젓가락까지…피해 아동 “학교엔 가고 싶다”","부모, 모진, 학대, 경남, 창녕, 초등, 학년, 젓가락, 발바닥, 학대, 추가, ...",2,0.226850


In [165]:
tmp_df['cluster_label'] = cluster_label

In [166]:
tmp_df[tmp_df['cluster_label']==0].sort_values(by='date')

,title,text,url,media,date,title_V,text_V,cluster_label,silhouette_coeff
2205,"‘중국 우한 폐렴’ 국내 첫 발생, 위기경보 ‘주의’로",ㆍ19일 입국 30대 중국 국적 여성\n\n‘중국 우한 폐렴’ 확진자가 국내에서 처...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-21,"‘중국 우한 폐렴’ 국내 첫 발생, 위기경보 ‘주의’로","입국, 중국, 국적, 여성, 중국, 우한, 폐렴, 진자, 국내, 처음, 보건, 당국...",0,0.392123
2211,"‘우한 폐렴’, “사람 간 전염 가능한 듯…전염력 크기는 지켜봐야”","ㆍ확산 속도는…신종 질병이라 파악 단계\nㆍ메르스 사태와 비교 어려워\nㆍ질본, 빠...",https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-21,"‘우한 폐렴’, “사람 간 전염 가능한 듯…전염력 크기는 지켜봐야”","확산, 속도, 신종, 질병, 파악, 단계, 메르스, 사태, 비교, 검사, 법, 개발...",0,0.395494
2299,"우한 폐렴은 신종 코로나바이러스, 공기로 전파 여부는 확인 안돼",[오늘의 세상]\n메르스·사스와 다른 7번째 신종… 중국서 가족끼리 감염사례 나와\...,https://news.naver.com/main/read.nhn?mode=LPOD...,조선일보,2020-01-21,"우한 폐렴은 신종 코로나바이러스, 공기로 전파 여부는 확인 안돼","오늘, 세상, 메르스, 사스, 신종, 중국, 가족, 끼리, 감염, 사례, 분비물, ...",0,0.539985
2367,"우한 폐렴, 사람간 전파 확인… 中 대확산 1~2주가 고비",중국 당국이 신종 코로나바이러스감염증(우한 폐렴)의 사람 간 전파를 인정하고 의료진...,https://news.naver.com/main/read.nhn?mode=LPOD...,국민일보,2020-01-22,"우한 폐렴, 사람간 전파 확인… 中 대확산 1~2주가 고비","중국, 당국, 신종, 코로나바이러스, 감염증, 우한, 폐렴, 사람, 전파, 인정, ...",0,0.526452
2447,"우한폐렴 사람간 전파 확인, 中의료진도 20명 감염","중국 감염자 309명·사망자 6명\n""북한, 中관광객 입국 잠정 금지""\n\n\n중...",https://news.naver.com/main/read.nhn?mode=LPOD...,조선일보,2020-01-22,"우한폐렴 사람간 전파 확인, 中의료진도 20명 감염","중국, 감염, 사망자, 북한, 관광객, 입국, 잠정, 금지, 중국, 우한, 폐렴, ...",0,0.234567
...,...,...,...,...,...,...,...,...,...
28941,"중증환자 수용 병상, 더이상 여유 없다",수도권 여유 병상 10개 불과 \n\n앞으로 5일내 병상 동날듯\n\n신종 코로나바...,https://news.naver.com/main/read.nhn?mode=LPOD...,문화일보,2020-08-31,"중증환자 수용 병상, 더이상 여유 없다","수도권, 여유, 병상, 불과, 앞, 병상, 날듯, 신종, 코로나바이러스, 감염증, ...",0,0.452233
28944,학원·독서실 모두 문닫아… 공부할 곳 잃은 학생들,“집중해 공부할곳 없다” 하소연\n\n공부방·과외 등에 학생 몰려\n\n신종 코로나...,https://news.naver.com/main/read.nhn?mode=LPOD...,문화일보,2020-08-31,학원·독서실 모두 문닫아… 공부할 곳 잃은 학생들,"집중, 공부, 하소연, 부방, 과외, 학생, 신종, 코로나바이러스, 감염증, 코로나...",0,0.133052
28945,"민주당, 2차 재난지원금 지급·4차 추경 논의 가속도",이르면 내일 고위 당정청 개최\n\n추석 민생대책으로 내놓을수도\n\n더불어민주당이...,https://news.naver.com/main/read.nhn?mode=LPOD...,문화일보,2020-08-31,"민주당, 2차 재난지원금 지급·4차 추경 논의 가속도","내일, 고위, 정청, 개최, 추석, 민생, 대책, 민주당, 재난, 원금, 지급, 추...",0,0.392123
28922,민주당 새 대표 이낙연 “코로나 국난 극복할 것”,더불어민주당 새 당 대표에 이낙연 의원(68·사진)이 선출됐다. 문재인 정부 초대 ...,https://news.naver.com/main/read.nhn?mode=LPOD...,동아일보,2020-08-31,민주당 새 대표 이낙연 “코로나 국난 극복할 것”,"민주당, 대표, 이낙연, 의원, 사진, 선출, 문재인, 정부, 초대, 국무총리, 이...",0,0.341992


In [167]:
tmp_df[tmp_df['cluster_label']==1].sort_values(by='date')

,title,text,url,media,date,title_V,text_V,cluster_label,silhouette_coeff
0,무너지는 일과 삶의 경계···노동이 녹아내린다,이 땅에서 ‘비정규직’이라는 표현이 광범위하게 쓰인 지 20여년. 정부가 신규 일자...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01,무너지는 일과 삶의 경계···노동이 녹아내린다,"비정규직, 표현, 정부, 신규, 일자리, 비정규직, 정규직, 전환, 주력, 사이, ...",1,0.475759
84,"원유 매장량 세계 1위 베네수엘라, 주유소마다 대기 차량 장사진","[적대사회, 신음하는 지구촌] “정부 밀매 위해 공급 조절” 관측도\n\n\n\n“...",https://news.naver.com/main/read.nhn?mode=LPOD...,한국일보,2020-01-01,"원유 매장량 세계 1위 베네수엘라, 주유소마다 대기 차량 장사진","적대, 사회, 신음, 지구촌, 정부, 공급, 조절, 관측, 어제, 기름, 두, 원유...",1,0.387279
83,차베스 ‘국유화 광풍’… 측근들 기업 수탈에 산업도시가 유령도시로,"[적대사회, 신음하는 지구촌] <1> 국민이 등돌린 베네수엘라\n\n1300여곳 국...",https://news.naver.com/main/read.nhn?mode=LPOD...,한국일보,2020-01-01,차베스 ‘국유화 광풍’… 측근들 기업 수탈에 산업도시가 유령도시로,"적대, 사회, 신음, 지구촌, 국민, 베네수엘라, 국유화, 제조업, 메카, 마라카이...",1,0.431873
77,총선 4개월 앞… 야당심판론이 정권심판론 앞섰다,"본보, 한국리서치와 여론조사\n\n\n\n 4월 15일 실시되는 21대 총선을 10...",https://news.naver.com/main/read.nhn?mode=LPOD...,한국일보,2020-01-01,총선 4개월 앞… 야당심판론이 정권심판론 앞섰다,"한국, 리서치, 여론조사, 실시, 총선, 민심, 정권, 심판, 야당, 심판, 기운,...",1,0.345602
76,"[한국일보 1면 사진] 적대사회, 이미 닥친 디스토피아 (1월 1일자)",지난달 12일 베네수엘라 수도 카라카스 중심지 볼리바르 광장 인근 관공서에 연말을 ...,https://news.naver.com/main/read.nhn?mode=LPOD...,한국일보,2020-01-01,"[한국일보 1면 사진] 적대사회, 이미 닥친 디스토피아 (1월 1일자)","지난달, 베네수엘라, 수도, 카라카스, 중심지, 볼리바르, 광장, 인근, 관공서, ...",1,0.472709
...,...,...,...,...,...,...,...,...,...
28958,“이번 주 확산세 진정 안 되면 3단계로” 방역당국 ‘마지막 카드’ 남기고 배수진,30일부터 수도권에서 사회적 거리두기 2.5단계가 시행되면서 정부가 3단계 격상 여...,https://news.naver.com/main/read.nhn?mode=LPOD...,서울신문,2020-08-31,“이번 주 확산세 진정 안 되면 3단계로” 방역당국 ‘마지막 카드’ 남기고 배수진,"수도권, 사회, 거리, 두기, 단계, 시행, 정부, 단계, 격상, 결정, 앞서, 모...",1,0.166745
28962,“2차 재난지원금 빨리 풀어야” 커지는 목소리,수도권 내 거리두기 2.5단계 시행으로 소상공인과 자영업자에 심각한 피해가 우려되면...,https://news.naver.com/main/read.nhn?mode=LPOD...,서울신문,2020-08-31,“2차 재난지원금 빨리 풀어야” 커지는 목소리,"수도권, 거리, 두기, 단계, 시행, 상공, 자영, 업자, 피해, 우려, 정부, 긴...",1,0.209380
28963,“비대위 다수 파업 반대” 내부 폭로… 대전협 “절차상 문제 없어”,전공의들이 30일 무기한 집단휴진(파업) 등 단체행동을 지속하기로 하면서 논란이 가...,https://news.naver.com/main/read.nhn?mode=LPOD...,서울신문,2020-08-31,“비대위 다수 파업 반대” 내부 폭로… 대전협 “절차상 문제 없어”,"전공, 무기한, 집단, 파업, 단체행동, 지속, 논란, 가열, 의협, 의회, 대전,...",1,0.429428
28965,“응급 환자 돌보지 않을 만큼 정당합니까”… 의대생들 소신 발언,“정부안 공공의료 강화 미비하지만\n대안 없는 증원 반대 공감 못 얻어”\n\n“의...,https://news.naver.com/main/read.nhn?mode=LPOD...,서울신문,2020-08-31,“응급 환자 돌보지 않을 만큼 정당합니까”… 의대생들 소신 발언,"정부, 의료, 강화, 대안, 증원, 반대, 공감, 의사, 파업, 파업, 병원, 응급...",1,0.339777


In [168]:
tmp_df[tmp_df['cluster_label']==2].sort_values(by='date')

,title,text,url,media,date,title_V,text_V,cluster_label,silhouette_coeff
1,일자리 아닌 일거리···'일정치 않음'을 살다 [녹아내리는 노동],ㆍ①일자리보다 일거리\n\n노동의 미래를 모두 알 수는 없다. 다만 현실에서 일어나...,https://news.naver.com/main/read.nhn?mode=LPOD...,경향신문,2020-01-01,일자리 아닌 일거리···'일정치 않음'을 살다 [녹아내리는 노동],"일자리, 일거리, 노동, 미래, 현실, 변화, 사회, 준비, 짐작, 노동, 미래, ...",2,0.121173
41,오피스텔·상업용 건물 기준시가 소폭 오른다,"국세청, 각각 1.36%·2.39% 고시\n\n 오피스텔과 상업용 건물에 대한 ...",https://news.naver.com/main/read.nhn?mode=LPOD...,세계일보,2020-01-01,오피스텔·상업용 건물 기준시가 소폭 오른다,"국세청, 고시, 오피스텔, 상업, 건물, 대한, 상속, 증여, 양도소득세, 활용, ...",2,0.134758
42,이건희 회장 최고 주식부호… 증가액도 1등,1년간 4조원 늘어… 17조원 보유 / 지분총액 이재용·서경배·정몽구 / 국민연금 ...,https://news.naver.com/main/read.nhn?mode=LPOD...,세계일보,2020-01-01,이건희 회장 최고 주식부호… 증가액도 1등,"조원, 조원, 보유, 분, 액, 이재용, 경배, 정몽구, 국민연금, 보유, 삼성, ...",2,0.050153
43,‘親與 공수처장’ 뻔한데… 입맛대로 수사 ‘정권의 칼’ 우려 [공수처법 통과 이후],수장 임명싸고 정치권 재충돌 예고 / 추천위원 7명 중 다수가 친여 성향 / 대통령...,https://news.naver.com/main/read.nhn?mode=LPOD...,세계일보,2020-01-01,‘親與 공수처장’ 뻔한데… 입맛대로 수사 ‘정권의 칼’ 우려 [공수처법 통과 이후],"수장, 임명, 정치권, 충돌, 예고, 추천, 위원, 친, 성향, 대통령, 후보, 명...",2,0.031725
44,공수처발 한파에 국회 올스톱 위기... 총리 인준·민생입법 어쩌나 [공수처법 통과 이후],식물·동물 이어 ‘표류국회’ 되나 / 한국당 對與투쟁 강화·보수 통합 박차 / 홍준...,https://news.naver.com/main/read.nhn?mode=LPOD...,세계일보,2020-01-01,공수처발 한파에 국회 올스톱 위기... 총리 인준·민생입법 어쩌나 [공수처법 통과 이후],"식물, 동물, 표류, 국회, 한국, 투쟁, 강화, 보수, 통합, 박차, 홍준표, 통...",2,0.141513
...,...,...,...,...,...,...,...,...,...
28943,블라인드 치고 꼼수영업… 편의점앞 술판… 대면예배 고수,"- 곳곳서 방역수칙 위반\n\n편의점앞 자리없어 돌아가기도\n\n서울시, 음식점 특...",https://news.naver.com/main/read.nhn?mode=LPOD...,문화일보,2020-08-31,블라인드 치고 꼼수영업… 편의점앞 술판… 대면예배 고수,"곳곳, 방역, 칙, 위반, 편의점, 앞, 자리, 서울시, 음식점, 특별, 전수, 점...",2,0.216662
28953,"최재형, 與의 동서 거론에 “가족이 감사원 일하는 것 아니다”",양이원영 질의에 조목조목 반박\n\n독립성·중립성 침해 우려 일축\n\n더불어민주당...,https://news.naver.com/main/read.nhn?mode=LPOD...,문화일보,2020-08-31,"최재형, 與의 동서 거론에 “가족이 감사원 일하는 것 아니다”","이원영, 질의, 조목조목, 반박, 독립성, 중립성, 침해, 우려, 민주당, 여권, ...",2,0.033334
28954,"일상, 일주일만 멈춥시다",코로나19가 사실상 대한민국의 심장부를 멈춰 세웠다. 30일 0시를 기해 서울과 경...,https://news.naver.com/main/read.nhn?mode=LPOD...,서울신문,2020-08-31,"일상, 일주일만 멈춥시다","코로나, 사실, 대한민국, 심장, 기해, 서울, 경기, 인천, 수도권, 사회, 거리...",2,-0.055011
28913,잇단 마스크 시비… 착용 요구 승객 폭행한 50대 구속,신종 코로나바이러스감염증(코로나19) 확산세가 지속되면서 마스크 착용을 둘러싼 시비...,https://news.naver.com/main/read.nhn?mode=LPOD...,국민일보,2020-08-31,잇단 마스크 시비… 착용 요구 승객 폭행한 50대 구속,"신종, 코로나바이러스, 감염증, 코로나, 확산, 지속, 마스크, 착용, 시비, 폭행...",2,-0.091978


In [169]:
# tmp_df[tmp_df['cluster_label']==3].sort_values(by='date')

In [170]:
# tmp_df[tmp_df['cluster_label']==4].sort_values(by='date')

In [171]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명들을 반환함. 
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.  
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화. 
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함. 
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함 
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['text_V']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
        
    return cluster_details

In [172]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
#         print('Reviews 파일명 :',cluster_detail['filenames'][:7])
        print('==================================================')

In [173]:
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=tmp_df,\
                                  feature_names=feature_names, clusters_num=3, top_n_features=5)
# top_features : 중심과 가까운 값
print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['코로나', '정부', '상황', '서울', '지난']
####### Cluster 1
Top features: ['정부', '대한', '금지', '상황', '서울']
####### Cluster 2
Top features: ['금지', '서울', '배포', '무단', '대한']


- Cluster 0 : 정치
- Cluster 1 : 사회
- Cluster 2 : 경제

In [174]:
score_samples = silhouette_samples(feature_vect, tmp_df['cluster_label'])
print('silhouette_samples return 값의 shape' , score_samples.shape)

silhouette_samples return 값의 shape (29041,)


In [175]:
tmp_df['silhouette_coeff'] = score_samples

In [176]:
# 모든 데이터의 평균 실루엣 계수를 구함
average_score = silhouette_score(feature_vect, tmp_df['cluster_label'])

In [177]:
print('뉴스 타이틀 Silhouette Analysis Score:{0:.3f}'.format(average_score))
tmp_df['silhouette_coeff']

뉴스 타이틀 Silhouette Analysis Score:0.241


0        0.475649
1        0.121188
2        0.012159
3        0.488819
4        0.187640
           ...   
29036    0.118647
29037   -0.072879
29038    0.140636
29039    0.479585
29040    0.366266
Name: silhouette_coeff, Length: 29041, dtype: float64

### 모델링